# **Lexicon**

In [1]:
import numpy as np
import pandas as pd

### Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### DB 연결
# !pip install pymysql
import pymysql

### DB에 저장
import sqlalchemy
from sqlalchemy import create_engine

### 실시간 주식가격 데이터
# !pip install finance-datareader
import FinanceDataReader as fdr

### 텍스트 분석
## KoNLPy
# 1) JAVA 설치, 2) Python 버전과 맞는 JPype1-py3 설치, 3) !pip install konlpy, 4) 설치 경로에서 jvm.py 파일 코드 67번 줄 주석 처리 
from konlpy.tag import Okt
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
## FastText
# !pip install gensim
# !pip install fasttext
import fasttext
import fasttext.util
# Facebook 한국어 Embedding 모델 다운로드 → 한 번만 설치하면 됨
fasttext.util.download_model('ko', if_exists='ignore')
# 유사도 계산
from gensim import models

### 모델 학습 및 평가
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import precision_score, recall_score, f1_score

### 모델 저장 및 로드
import joblib

### 기타
import datetime
from collections import Counter
import sys
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf

print('GPU', '사용 가능' if tf.config.experimental.list_physical_devices('GPU') else '사용 불가능')

GPU 사용 불가능


In [3]:
!nvidia-smi

Sun Sep 19 03:45:08 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 452.56       Driver Version: 452.56       CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce MX450      WDDM  | 00000000:2D:00.0 Off |                  N/A |
| N/A   62C    P8    N/A /  N/A |    119MiB /  2048MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## **Read KOSELF & Test Data**

### **① KOSELF 감성 어휘 사전**

In [4]:
# 블로그에서 가져온 기본적인 한국어 긍부정 텍스트 목록
with open('positive_words_self.txt', encoding='utf-8') as pos_blog:
    positive_blog = pos_blog.readlines()
positive_blog = [pos_blog.replace('\n', '') for pos_blog in positive_blog]
with open('negative_words_self.txt', encoding='utf-8') as neg_blog:
    negative_blog = neg_blog.readlines()
negative_blog = [neg_blog.replace('\n', '') for neg_blog in negative_blog]

# KOSELF 감성 어휘 사전
with open('KOSELF_pos.txt', encoding='utf-8') as pos:
    positive = pos.readlines()
positive = [pos.replace('\n', '') for pos in positive]
with open('KOSELF_neg.txt', encoding='utf-8') as neg:
    negative = neg.readlines()
negative = [neg.replace('\n', '') for neg in negative]

### **② 2018년 LG Data**

In [5]:
lexicon_2018 = pd.read_csv('../../../../Code/Data/Test/Stock-Year/2018_lg.csv')

lexicon_2018.head(1)

,st_n,st_cd,news,datetime,title,url,text,date,time,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed,Tokenization
0,LG화학,51910,매일경제,2018010118,中정부 전기차 면세 연장에 코스모화학 등 수혜株 주목,http://news.mk.co.kr/newsRead.php?year=2018&no...,\n\n\n 중국발 수요 증가에 힘입어 새해 '전기차 랠리'에 대한 기대감이 확산되...,2018-01-02,18,407000,415000,406500,411000,153739,0.014815,1,0,중국 수요 증가 새해 전기차 랠리 대한 대감 확산 한국 거래소 전기차 배터리 핵심 ...


In [6]:
lexicon_2018[lexicon_2018['text'].isna()]

,st_n,st_cd,news,datetime,title,url,text,date,time,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed,Tokenization


In [7]:
lexicon_2018.dropna(axis=0, inplace=True)
lexicon_2018 = lexicon_2018.reset_index(drop=True)
lexicon_2018.isna().sum()

st_n                 0
st_cd                0
news                 0
datetime             0
title                0
url                  0
text                 0
date                 0
time                 0
Open                 0
High                 0
Low                  0
Close                0
Volume               0
Change               0
UpDown               0
Extremely_Changed    0
Tokenization         0
dtype: int64

## **Calculate Cosine Similarity**

In [8]:
# 이미 학습된 FastText 내장 한국어 모델
try:
    print(ko_model, '모델 로드 불필요')
except:
    ko_model = models.fasttext.load_facebook_model('cc.ko.300.bin')

In [9]:
limit_number = 15000
sys.setrecursionlimit(limit_number)

In [10]:
neg_lexicon_2018 = pd.DataFrame(columns=['date', 'news_num', 'KOSELF_neg_word', 'news_word', 'cosine_similarity', 'frequency'])   # 'frequency' 구현할 방법은 추후 생각...ㅠㅠ

# KOSELF_neg와의 Cosine Similarity 계산
for x in range(len(lexicon_2018['Tokenization'])):
    news_num = x+1
    for y in range(len(negative)):
        for z in range(len(list(set(lexicon_2018['Tokenization'][x].split())))):
            if (ko_model.wv.similarity(negative[y], list(set(lexicon_2018['Tokenization'][x].split()))[z]) >= 0.5) and (ko_model.wv.similarity(negative[y], list(set(lexicon_2018['Tokenization'][x].split()))[z]) != 1.0):
                freq = 0
                for w in range(len(lexicon_2018['Tokenization'][x].split())):
                    if lexicon_2018['Tokenization'][x].split()[w] == list(set(lexicon_2018['Tokenization'][x].split()))[z]:
                        freq += 1
                data = {
                    'date': lexicon_2018['date'][x],
                    'news_num': news_num,
                    'KOSELF_neg_word': negative[y],
                    'news_word': list(set(lexicon_2018['Tokenization'][x].split()))[z],
                    'cosine_similarity': ko_model.wv.similarity(negative[y], list(set(lexicon_2018['Tokenization'][x].split()))[z]),
                    'frequency': freq
                    }
                neg_lexicon_2018 = neg_lexicon_2018.append(data, ignore_index=True)
                print("***{0} Cosine Similarity between <{1}> & <{2}> : {3}".format(lexicon_2018['date'][x], negative[y], list(set(lexicon_2018['Tokenization'][x].split()))[z], ko_model.wv.similarity(negative[y], list(set(lexicon_2018['Tokenization'][x].split()))[z])))

***2018-01-02 Cosine Similarity between <둔화> & <증가> : 0.5041859149932861
***2018-01-02 Cosine Similarity between <하락> & <증가> : 0.553473711013794
***2018-01-02 Cosine Similarity between <하락> & <상승> : 0.7126480340957642
***2018-01-02 Cosine Similarity between <과적> & <차량> : 0.5693626403808594
***2018-01-02 Cosine Similarity between <둔화> & <증가> : 0.5041859149932861
***2018-01-02 Cosine Similarity between <하락> & <증가> : 0.553473711013794
***2018-01-02 Cosine Similarity between <과적> & <차량> : 0.5693626403808594
***2018-01-02 Cosine Similarity between <둔화> & <증가> : 0.5041859149932861
***2018-01-02 Cosine Similarity between <약화> & <강화> : 0.5141976475715637
***2018-01-02 Cosine Similarity between <침체> & <위기> : 0.5072819590568542
***2018-01-02 Cosine Similarity between <하락> & <증가> : 0.553473711013794
***2018-01-02 Cosine Similarity between <약화> & <강화> : 0.5141976475715637
***2018-01-03 Cosine Similarity between <둔화> & <반등> : 0.5092458128929138
***2018-01-03 Cosine Similarity between <둔화> & <하락> : 

In [11]:
neg_lexicon_2018

,date,news_num,KOSELF_neg_word,news_word,cosine_similarity,frequency
0,2018-01-02,1,둔화,증가,0.504186,1
1,2018-01-02,1,하락,증가,0.553474,1
2,2018-01-02,1,하락,상승,0.712648,2
3,2018-01-02,2,과적,차량,0.569363,1
4,2018-01-02,2,둔화,증가,0.504186,1
...,...,...,...,...,...,...
7992,2018-12-28,1707,하락,상승,0.712648,6
7993,2018-12-28,1707,하락,급등,0.614909,1
7994,2019-01-02,1708,둔화,감소,0.534909,1
7995,2019-01-02,1708,약화,감소,0.551185,1


In [12]:
neg_lexicon_2018.to_csv('../../../../Code/Data/Test/Stock-Year/neg_lg_2018.csv', index=False)

In [13]:
a = list(set(list(neg_lexicon_2018['news_word'])))

a

['업황',
 '운항',
 '악화',
 '장기물량',
 '불황',
 '폐업',
 '파산보호',
 '차량',
 '증가',
 '보상',
 '호전',
 '악재',
 '조처',
 '실패',
 '보합',
 '선박',
 '강화',
 '견조',
 '좌절',
 '폭락',
 '경색',
 '금수',
 '여준호',
 '병세',
 '위축',
 '약세',
 '등락',
 '상승',
 '급감',
 '반등',
 '당혹',
 '급락',
 '의심',
 '특혜',
 '절상',
 '부도',
 '꼽으셨는데',
 '질책',
 '위기',
 '약화',
 '플러스',
 '손해배상',
 '경기',
 '호황',
 '하락',
 '유발',
 '침체',
 '끝내',
 '낙석',
 '역풍',
 '화물',
 '주춤',
 '비리',
 '혼돈',
 '강점',
 '저하',
 '급등',
 '둔화',
 '성공',
 '스캔들',
 '폭등',
 '불안',
 '논란',
 '트라젠타',
 '우발',
 '항행',
 '감소']

In [14]:
b = list(set(list(neg_lexicon_2018[neg_lexicon_2018['cosine_similarity']>=0.7]['news_word'])))

b

['실패', '약화', '침체', '성공', '스캔들', '상승', '의심']

In [15]:
c = list(set(list(neg_lexicon_2018[neg_lexicon_2018['cosine_similarity']>=0.65]['news_word'])))

c

['실패', '약화', '침체', '성공', '스캔들', '상승', '급락', '의심', '조처']